In [1]:
import numpy as np
from astropy.table import Table
from query_galaxies_as_deflectors import find_potential_lenses_parallel
from query_galaxies_as_deflectors import find_potential_lenses
from astropy.cosmology import FlatLambdaCDM
import srclensprop as slp
import os
from slsim.Util.mag2errors import get_errors_Poisson
from mpi4py import MPI

This notebook demonstrates how to use the 'query_galaxies_as_deflectors.py' module in slsim to query a galaxy catalog and idenitfy which of the galaxies can act as potential deflectors.  
Please download the catalogs 'cosmo_DC2.fits' from 'https://github.com/LSST-strong-lensing/data_public' for running this notebook.

In [2]:
# define the cosmology to be used
cosmo = FlatLambdaCDM(H0=72, Om0=0.26)

# define the Universal gravitational constant and the speed of light
constants = {
    "G": 4.2994e-9,
    "light_speed": 299792.458,
}  # in km^3 Msun^(-1) year^(-2) and km/s respectively

deflector_type = "galaxy"
# background_source = "galaxy"
# background_source = "quasar"

In [3]:
# Define path to the input catalog
lenscatalog = "cosmo_DC2.fits"  ## foreground lens catalog


# define the keywords for the survey, deflector, and the background source to be used
kwargs_source_gal = {
    "type": "galaxy",
    "min_z": 0.2,
    "max_z": 5.0,
    "min_mag": 23.0,
    "max_mag": 28.0,
    "boost_csect": 3,
    "ell_min": 0.1,
    "ell_max": 0.8,
    "PA_min": 0.0,
    "PA_max": 180.0,
    "R_Einst_min": 0.5,
    "R_Einst_max": 3.0,
}
kwargs_source_qso = {
    "type": "qso",
    "min_z": 0.2,
    "max_z": 5.0,
    "min_mag": 23.0,
    "max_mag": 28.0,
    "boost_csect": 30000,
    "ell_min": None,
    "ell_max": None,
    "PA_min": None,
    "PA_max": None,
    "R_Einst_min": 0.9,
    "R_Einst_max": 3.0,
}
kwargs_deflector = {
    "min_z": 0.2,
    "max_z": 1.9,
    "min_shear": 0.001,
    "max_shear": 0.02,
    "min_PA_shear": 0.0,
    "max_PA_shear": 180.0,
}

Please make sure that the input galaxy ctalog has the required columnnames, or else change the column names as shown below.

In [4]:
# read the foreground galaxy catalog as an astropy table
foreground_table = Table.read(lenscatalog, format="fits")

# sort the foreground table for redshift
id_redshift = np.where(foreground_table["redshift"] < 2)[0]
foreground_table = foreground_table[id_redshift]
print("Size of foreground table: ", len(foreground_table))

# rename the column names
foreground_table.rename_column("mag_true_g_lsst", "gmag")
foreground_table.rename_column("mag_true_r_lsst", "rmag")
foreground_table.rename_column("mag_true_i_lsst", "imag")
foreground_table.rename_column("ellipticity_true", "ellipticity")

# rename the column name and generate the random shear if not available
if "shear" and "shear PA" in foreground_table.colnames:
    foreground_table.rename_column("shear", "random shear")
    foreground_table.rename_column("shear PA", "random shear PA")

else:
    # generate random shear to the galaxies if shear is not already provided
    myseed = 2894  # 9999
    np.random.seed(myseed)
    random_shear_value = np.random.uniform(
        low=kwargs_deflector.get("min_shear"), high=kwargs_deflector.get("max_shear")
    )
    # print('random_shear_value is :',random_shear_value)
    random_shear = np.full(len(foreground_table), random_shear_value)
    # random_shear=np.random.uniform(low=kwargs_deflector.get('min_shear'),high=kwargs_deflector.get('max_shear'),size=len(foreground_table))

    np.random.seed(myseed)
    random_PA_value = np.random.uniform(
        kwargs_deflector.get("min_PA_shear"), kwargs_deflector.get("max_PA_shear")
    )
    random_shear_PA = np.full(len(foreground_table), random_PA_value)
    # random_shear_PA = np.random.uniform(kwargs_deflector.get('min_PA_shear'),kwargs_deflector.get('max_PA_shear'),size=len(foreground_table))

    # add the shear and shear PA to the galaxy catalog table
    foreground_table["random shear"] = random_shear
    foreground_table["random shear PA"] = random_shear_PA

Size of foreground table:  583402


If the input catalog doesnot contain magnitude errors, please add statistical error due to Poisson noise in the counts.

In [5]:
# define the LSST zeropoint and exposure time (in secs) for g, r, and i bands.
zeropoint_g, exptime_g = 28.51, 15
zeropoint_r, exptime_r = 28.36, 15
zeropoint_i, exptime_i = 28.17, 15


# Add magnitude errors (Poisson) to the table if not already present
if "err_g" and "err_r" and "err_i" in foreground_table.colnames:
    pass
else:
    foreground_table["err_g"] = get_errors_Poisson(
        foreground_table["gmag"], zeropoint_g, exptime_g
    )
    foreground_table["err_r"] = get_errors_Poisson(
        foreground_table["rmag"], zeropoint_r, exptime_r
    )
    foreground_table["err_i"] = get_errors_Poisson(
        foreground_table["imag"], zeropoint_i, exptime_i
    )

Please create the Crosssect.dat file, if not already created

In [6]:
# Create Crosssect.dat file, if not already created
filename = "Crosssect.dat"

if os.path.exists(filename):
    print(f"{filename} already exists, exiting without creating a new file.")
else:
    print(f"File {filename} does not exist, creating it now.")

    qi = np.arange(0.1, 1.0, 0.001)
    csecti = np.zeros_like(qi)

    for ii in range(qi.size):
        csecti[ii] = slp.getcrosssect(slp.fid_b_I / np.sqrt(qi[ii]), qi[ii])

    np.savetxt(filename, np.transpose([qi, csecti]))
    print(f"{filename} created successfully.")

Crosssect.dat already exists, exiting without creating a new file.


In [7]:
# Run the code in series
run = "series"

## To run in series
if run == "series":
    find_potential_lenses(
        foreground_table[0:50000],
        background_source="galaxy",
        kwargs_source=kwargs_source_gal,
        kwargs_deflector=kwargs_deflector,
        cosmo=cosmo,
        constants=constants,
    )

Initializing Cross-section
Found  822 unique lensed  galaxy


To deal with large catalogs, the code can also be run in parallel,  
To run the code in parallel, save the code as a python script *.py  
'Openmpi' should be setup in your machine and at the top, import mpi4py as:  
 from mpi4py import MPI

--Then run as 'mpirun -np X python query_galaxies_as_deflectors.py',  

-- where X is the no. of processors you want to use.  




In [1]:
# run='parallel'

if run == "parallel":
    ## MPI Initialization
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    find_potential_lenses_parallel(
        comm,
        rank,
        size,
        foreground_table[0:50000],
        background_source="galaxy",
        kwargs_source=kwargs_source_gal,
        kwargs_deflector=kwargs_deflector,
        cosmo=cosmo,
        constants=constants,
    )

else:
    print("Invalid input")

NameError: name 'run' is not defined